# Show3DVolume: Orthogonal Slice Viewer
Demonstrates the `Show3DVolume` widget for viewing 3D volumetric data with XY, XZ, and YZ slice planes.

In [ ]:
%env ANYWIDGET_HMR=1

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass  # autoreload unavailable (Colab Python 3.12+)

## 1. Crystal lattice with atoms
Gaussian atoms arranged in a BCC-like lattice -- realistic for TEM volume data.

In [ ]:
N = 64
vol = np.zeros((N, N, N), dtype=np.float32)
zz, yy, xx = np.mgrid[:N, :N, :N]
# BCC lattice: corner + body-center atoms
spacing = 16
sigma = 2.5
for iz in range(0, N, spacing):
    for iy in range(0, N, spacing):
        for ix in range(0, N, spacing):
            # Corner atom
            r2 = (zz - iz)**2 + (yy - iy)**2 + (xx - ix)**2
            vol += np.exp(-r2 / (2 * sigma**2))
            # Body-center atom
            cz, cy, cx = iz + spacing // 2, iy + spacing // 2, ix + spacing // 2
            if cz < N and cy < N and cx < N:
                r2 = (zz - cz)**2 + (yy - cy)**2 + (xx - cx)**2
                vol += np.exp(-r2 / (2 * sigma**2))
                
# zz, yy, xx = np.mgrid[:N, :N, :N]
Show3DVolume(vol, title='BCC crystal lattice', cmap='inferno')

## 2. Random atom cluster with noise
Scattered atoms with varying intensities in a noisy background -- tests auto contrast.

In [ ]:
np.random.seed(7)
N = 64
vol = np.random.randn(N, N, N).astype(np.float32) * 0.05
zz, yy, xx = np.mgrid[:N, :N, :N]
# Random atom positions
n_atoms = 40
positions = np.random.rand(n_atoms, 3) * (N - 10) + 5
intensities = np.random.uniform(0.5, 1.0, n_atoms)
sigmas = np.random.uniform(1.5, 3.0, n_atoms)
for i in range(n_atoms):
    pz, py, px = positions[i]
    r2 = (zz - pz)**2 + (yy - py)**2 + (xx - px)**2
    vol += intensities[i] * np.exp(-r2 / (2 * sigmas[i]**2))
Show3DVolume(vol, title='Atom cluster (noisy)', cmap='viridis', auto_contrast=True)

## 3. Nanoparticle with shell
Spherical nanoparticle with a denser core and lighter shell -- use playback to sweep through Z slices.

In [ ]:
np.random.seed(0)
N = 80
coords = np.linspace(-1, 1, N)
zz, yy, xx = np.meshgrid(coords, coords, coords, indexing='ij')
r = np.sqrt(xx**2 + yy**2 + zz**2)
# Shell + core structure
vol = np.zeros((N, N, N), dtype=np.float32)
vol[(r > 0.5) & (r < 0.7)] = 0.4  # shell
vol[r < 0.35] = 1.0  # dense core
# Add atom-like texture inside the core
n_atoms = 60
for _ in range(n_atoms):
    pos = np.random.randn(3) * 0.12
    r2 = (xx - pos[0])**2 + (yy - pos[1])**2 + (zz - pos[2])**2
    vol += 0.5 * np.exp(-r2 / (2 * 0.03**2))
vol += np.random.randn(N, N, N).astype(np.float32) * 0.02
Show3DVolume(vol, title='Nanoparticle (core-shell)', cmap='magma')